# Анализ тональности отзывов с предварительно собранной выборкой

Глазунов А.В.

Данные собраны с сайтов Отзовик, Irecommend, а также с одного из сайтов с рекомендациями mail.ru с использованием Scrapy. Тестовые данные предобработаны циклом в нормальную форму, что позволило с ними работать.

In [33]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# Any results you write to the current directory are saved as output.

/kaggle/input/test-good/data_test_good.csv
/kaggle/input/reviews-avg/reviews_dataset.csv
/kaggle/input/product-reviews-sentiment-analysis/sample_submission.csv
/kaggle/input/product-reviews-sentiment-analysis/test.csv


In [34]:
import pandas as pd
reviews_dataset = pd.read_csv("../input/reviews-avg/reviews_dataset.csv")
data_test_good = pd.read_csv("../input/test-good/data_test_good.csv")

In [35]:
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.linear_model import LogisticRegression, SGDClassifier
import numpy as np
import pandas as pd
from sklearn.model_selection import cross_val_score
from sklearn.pipeline import Pipeline
from sklearn.utils import shuffle 
from sklearn.naive_bayes import MultinomialNB
from sklearn.svm import LinearSVC
from sklearn.model_selection import GridSearchCV

import warnings
warnings.filterwarnings('ignore')

Подготовим обучающую выборку

In [36]:
df=reviews_dataset

In [37]:
df = shuffle(df,random_state=42) 
df

,title,text,rating,class,text_cleaned
1018,Сотовый телефон Servo 225,"<div class=""review-body description"" itemprop=...",НЕТ,0,купил тут я для отца звонилку на алиэкспрессе ...
642,Мобильный телефон Xiaomi Mi 9T Pro,"<p>Всем привет, мне не терпится поделиться с в...",рекомендует,1,"Всем привет, мне не терпится поделиться с вами..."
2241,Sony Xperia XA1,"Превосходный дизайн, высокое качество сборки, ...",5.0,1,"Превосходный дизайн, высокое качество сборки, ..."
1377,Сотовый телефон Fly E195,"<div class=""review-body description"" itemprop=...",НЕТ,0,Покупал телефон на подмену. Но даже в этом отн...
2221,Sony Xperia Z5 Compact,не большой и может все.нет.супер,5.0,1,не большой и может все.нет.супер
...,...,...,...,...,...
1638,Смартфон Beeline Smart Dual,"<div class=""review-body description"" itemprop=...",НЕТ,0,За такую цену (3200 с лишним рублей) не стоило...
1095,Смартфон Jeep F605,"<div class=""review-body description"" itemprop=...",НЕТ,0,"< big"" ""//i5.otzovik.com/2017/02/04/4454381//4..."
1130,Смартфон Теле2 Midi LTE,"<div class=""review-body description"" itemprop=...",НЕТ,0,"Сильно нагревается, глючит, не даёт зайти в go..."
1294,Смартфон Digma VOX v40 3G,"<div class=""review-body description"" itemprop=...",НЕТ,0,Скончался скоропостижно Леново и пришлось очен...


In [38]:
texts = df.dropna().text_cleaned
labels = df.dropna()['class']

Напишем функцию пайплайн для получения признакового описания данных и классификации

In [39]:
def text_classifier(vectorizer, classifier):
    return Pipeline(
            [('vectorizer', vectorizer),
             ('classifier', classifier)
            ])

Подберем векторайзер для логистической регрессии на бейзлайне

In [40]:
for vectorizer in [CountVectorizer,TfidfVectorizer]:
    print('{}'.format(vectorizer))
    print()
    print('Mean accuracy:')
    print(cross_val_score(text_classifier(vectorizer(),LogisticRegression()),
                          texts,labels,cv=5).mean())
    print('Standart deviation of accuracy:')
    print(cross_val_score(text_classifier(vectorizer(),LogisticRegression()),
                          texts,labels,cv=5).std())
    print()

<class 'sklearn.feature_extraction.text.CountVectorizer'>

Mean accuracy:
0.90311986863711
Standart deviation of accuracy:
0.013816541194064107

<class 'sklearn.feature_extraction.text.TfidfVectorizer'>

Mean accuracy:
0.9050903119868637
Standart deviation of accuracy:
0.005129884844602066



Подберем параметры для классификатора в первом приближении

In [ ]:
%%time
param_grid_Log_reg_Tfidf = {'vectorizer__ngram_range': [(1,1),(1,3),(1,4)],
              'vectorizer__min_df': [1, 10],
                'classifier__penalty':['l2','l1'],
                'classifier__C':[1,3]}
clf_Log_reg_Tfidf = GridSearchCV(text_classifier(TfidfVectorizer(),LogisticRegression(random_state=42)),
                             param_grid_Log_reg_Tfidf, n_jobs=-1)
clf_Log_reg_Tfidf.fit(texts, labels)
print("Best score found by grid search:")
print(clf_Log_reg_Tfidf.best_score_)
print(clf_Log_reg_Tfidf.best_estimator_)

Подберем параметр регуляризации

In [ ]:
param_grid_Log_reg_Tfidf2 = {
                     'classifier__C':[4,5,6,10]
                    }

In [ ]:

clf_Log_reg_Tfidf2 = GridSearchCV(text_classifier(TfidfVectorizer(ngram_range = (1,3),min_df=10),\
                            LogisticRegression(penalty='l2',random_state=42)),
                             param_grid_Log_reg_Tfidf2, n_jobs=-1)
clf_Log_reg_Tfidf2.fit(texts, labels)
print("Best score found by grid search:")
print(clf_Log_reg_Tfidf2.best_score_)
print(clf_Log_reg_Tfidf2.best_estimator_)   

Зададим все параметры из best_estimator_ вручную и проверим качество на кроссвалидации

In [ ]:
print(cross_val_score(text_classifier(TfidfVectorizer(ngram_range = (1,3),min_df=10),\
                            LogisticRegression(C=6,penalty='l2',random_state=42)),
                          texts,labels,cv=5).mean())

Заранее подготовленная тестовая выборка

In [ ]:
data_test_good 

Обучение модели и загрузка результатов в контест

In [ ]:
clf0 = text_classifier(TfidfVectorizer(ngram_range = (1,3),min_df=10),\
                            LogisticRegression(C=6,penalty='l2',random_state=42))
clf0.fit(texts,labels)

In [ ]:
y_pred0 = clf0.predict(data_test_good.values.T[0])
ans0 = pd.DataFrame({'Id' : np.arange(0, len(y_pred0)), 'y' : np.array(y_pred0)})
ans0.y=ans0['y'].apply(lambda x: 'pos' if x==1 else 'neg')
ans0.to_csv('../working/submission11.csv',sep=',', index=False)

Качество на несбалансированных данных на Kaggle получилось 0,94, что выше проходного 0,85.

Обучающая выборка несбалансирована, позитивных отзывов больше, сбалансируем и проверим качество

In [ ]:
import imblearn
from imblearn.over_sampling import RandomOverSampler
over_sampler = RandomOverSampler(random_state=0)

In [ ]:
X=texts.values.reshape(-1,1)
y=labels
X_over, y_over = over_sampler.fit_sample(X,y)
X_over = X_over.T[0]

Качество на кроссвалидации логистической регрессии на дополненных данных

In [ ]:
print(cross_val_score(text_classifier(TfidfVectorizer(ngram_range = (1,3),min_df=10),\
                            LogisticRegression(C=6,penalty='l2',random_state=42)),
                          X_over,y_over,cv=5).mean())

Обучение модели и загрузка результатов

In [ ]:
clf = text_classifier(TfidfVectorizer(ngram_range = (1,3),min_df=10),\
                            LogisticRegression(C=6,penalty='l2',random_state=42))
clf.fit(X_over,y_over)

In [ ]:
y_pred = clf.predict(data_test_good.values.T[0])
ans = pd.DataFrame({'Id' : np.arange(0, len(y_pred)), 'y' : np.array(y_pred)})
ans.y=ans['y'].apply(lambda x: 'pos' if x==1 else 'neg')
ans.to_csv('../working/submission2.csv',sep=',', index=False)

Точность на тесте в соревновании 0,90, что хуже предыдущего показателя на недополненных данных.

Попробуем линейный svm алгоритм с заранее выбранным параметром регуляризации на дополненных данных

In [ ]:
clf2 = text_classifier(TfidfVectorizer(ngram_range = (1,3),min_df=10),\
                            LinearSVC(C=4))
clf2.fit(X_over,y_over)

In [ ]:
y_pred2 = clf2.predict(data_test_good.values.T[0])
ans2 = pd.DataFrame({'Id' : np.arange(0, len(y_pred2)), 'y' : np.array(y_pred2)})
ans2.y=ans2['y'].apply(lambda x: 'pos' if x==1 else 'neg')
ans2.to_csv('../working/submission44.csv',sep=',', index=False)

Качество получилось 0,92, что хуже. Таким образом выбираем алгоритм с параметрами

Pipeline(memory=None,
         steps=[('vectorizer',
                 TfidfVectorizer(analyzer='word', binary=False,
                                 decode_error='strict',
                                 dtype=<class 'numpy.float64'>,
                                 encoding='utf-8', input='content',
                                 lowercase=True, max_df=1.0, max_features=None,
                                 min_df=10, ngram_range=(1, 3), norm='l2',
                                 preprocessor=None, smooth_idf=True,
                                 stop_words=None, strip_accents=None,
                                 sublinear_tf=False,
                                 token_pattern='(?u)\\b\\w\\w+\\b',
                                 tokenizer=None, use_idf=True,
                                 vocabulary=None)),
                ('classifier',
                 LogisticRegression(C=6, class_weight=None, dual=False,
                                    fit_intercept=True, intercept_scaling=1,
                                    l1_ratio=None, max_iter=100,
                                    multi_class='auto', n_jobs=None,
                                    penalty='l2', random_state=42,
                                    solver='lbfgs', tol=0.0001, verbose=0,
                                    warm_start=False))],
         verbose=False)

, обученный на несбалансированных данных. При этом дополнительно(не в этом ноутбуке) проверена эффективность лемматизации. Для логистической регрессии результатов не дал. Для других алгоритмов получилась даже просадка по качеству. Из 8590 признаков(слов в словаре) с помощью лемматизации удалось уменьшить это число до 8300.